# Hartmann Hahn polarising sequence

In [15]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [16]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [17]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [18]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV4'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [20]:
# for single frequency XY8-N:
hh_tau = OrderedDict()
hh_tau['mw_amp'] = 0.25
hh_tau['spinlock_amp'] = 0.1126
hh_tau['mw_freq'] = 100e6
hh_tau['tau_start'] = 1*1e-6
hh_tau['tau_incr'] = 15.0*1e-6
hh_tau['points'] = 20
hh_tau['measurement_time'] =40*60
hh_tau['refocus_interval'] = 2*60
hh_tau['alternating']=True
hh_tau['rabi_period']=159.6*1e-9

In [21]:
# generate new HH sequence
sequencegeneratorlogic.delete_sequence('HHtauseq')    
sequencegeneratorlogic.generate_Hartmann_Hahn_tau_sequence('HHtauseq', hh_tau['rabi_period'], hh_tau['spinlock_amp'], 100e6,
                   0.25, hh_tau['tau_start'], hh_tau['tau_incr'], hh_tau['points'],
                   'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', True)
# sample, upload and load waveform
if 'HHtauseq' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of HHtauseq in notebook timed out.')
pulsedmasterlogic.sample_sequence('HHtauseq', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [12]:
# # generate new HH sequence
# sequencegeneratorlogic.delete_sequence('HHahn_tau_seq')    
# sequencegeneratorlogic.generate_Hartmann_Hahn_tau_sequence('HHahn_tau_seq', hh_tau['rabi_period'], hh_tau['spinlock_amp'], 100e6,
#                    0.25, hh_tau['tau_start'], hh_tau['tau_incr'], hh_tau['points'],
#                    'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', True)
# # sample, upload and load waveform
# if 'HHahn_tau_seq' not in sequencegeneratorlogic.saved_pulse_sequences:
#     sequencegeneratorlogic.log.error('Sequence generation of HHtauseq in notebook timed out.')
# pulsedmasterlogic.sample_sequence('HHahn_tau_seq', True)

# while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(5)

# import winsound
# frequency = 300  # Set Frequency To 2500 Hertz
# duration = 1000  # Set Duration To 1000 ms == 1 second
# winsound.Beep(frequency, duration)

In [22]:
refocus_poi(setup['poi'])

163806.5 1.5799667646813502e-08


True

In [23]:
do_hh_tau_refocus(setup['poi'])

163806.5 1.1626825471672232e-08
163806.5 2.6850244729551215e-09
163806.5 3.005810816146228e-09
163806.5 1.3200870757738858e-09
163806.5 7.038121997686865e-09
163806.5 1.194493002202961e-08
163806.5 2.3904477421287013e-10
163806.5 4.171495845714475e-09
163806.5 2.7504296789289515e-09
163806.5 6.84322727305365e-09
163806.5 2.6292083627273046e-09
163806.5 1.0069229314824563e-08
163806.5 6.9102228445214e-09
163806.5 5.071802442048023e-09
163806.5 1.058369995143548e-08
163806.5 2.750909754475867e-09
163806.5 3.05695449709774e-09
163806.5 6.913173087381741e-09
163806.5 2.0639029174570396e-09


False

In [19]:
def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.getState() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_hh_tau(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    tau_arr = hh_tau['tau_start'] + np.arange(hh_tau['points']) * hh_tau['tau_incr']
    # set parameters in analysis tab
    if hh_tau['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*hh_tau['points'], 100e-6, [], hh_tau['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, hh_tau['points'], 100e-6, [], hh_tau['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hh_tau['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','SpinLocking-'+str(hh_tau['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_hh_tau_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    hh_tau_total_time = hh_tau['measurement_time']
    hh_tau_runtime = 0.0
    hh_tau['measurement_time'] = hh_tau['refocus_interval']
    end_measure = do_hh_tau('hh_tau_refocus','')
    hh_tau_runtime += hh_tau['refocus_interval']
    while hh_tau_total_time > hh_tau_runtime:
        for i in range(1):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_hh_tau('hh_tau_refocus', 'hh_tau_refocus')
        hh_tau_runtime += hh_tau['refocus_interval']
        if end_measure:
            break
    hh_tau['measurement_time'] = hh_tau_total_time
    # save measurement
    pulsedmasterlogic.save_measurement_data('s','SpinLocking'+'_'+str(hh_tau['spinlock_amp'])+'_'+setup['NV_name'],True)
    write_to_logfile('hh_tau_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=hh_tau_runtime)
    time.sleep(2)
    return end_measure

In [1]:
mw_freq = 1363.05e+6 # in MHz
mw_power = -10

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()